# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [2]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [14]:
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
response = requests.get(url)
data = response.json()
df = pd.json_normalize(data['items'])
df_rj = df[df['uf'] == 'RJ']



In [15]:
status_frequentes = df_rj['status'].value_counts().head(3)
print("\nTrês status mais frequentes e suas frequências:")
print(status_frequentes)


Três status mais frequentes e suas frequências:
status
Deferido        24
Arquivado       11
Regularizado    10
Name: count, dtype: int64


In [17]:
df_rj.loc[:, 'ano_status'] = df_rj['data_status'].str.slice(0, 4)

In [18]:
frequencia_ano_status = df_rj['ano_status'].value_counts()
print("\nFrequência de cada ano do campo construído:")
print(frequencia_ano_status)



Frequência de cada ano do campo construído:
ano_status
13/0    4
18/0    4
15/0    4
26/0    3
29/0    3
06/0    3
07/0    3
02/0    3
06/1    2
30/1    2
23/1    2
01/0    2
21/0    2
20/0    2
12/0    2
11/0    2
03/0    1
31/0    1
12/1    1
08/1    1
09/0    1
09/1    1
23/0    1
13/1    1
11/1    1
05/0    1
17/1    1
04/1    1
27/0    1
21/1    1
14/0    1
10/0    1
20/1    1
28/0    1
04/0    1
22/0    1
26/1    1
Name: count, dtype: int64


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [20]:
def consultar_api(uf, tipo_interessado):
    url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl"
    params = {
        'uf': uf,
        'interessado': tipo_interessado,
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data['items'])
        return df
    else:
        print("Erro na requisição. Código de status:", response.status_code)
        return None


In [22]:
# 2) Seu código aqui
df_mg_arquivado = consultar_api('MG', 'Estado')
solicitacoes_mg_arquivado = df_mg_arquivado[df_mg_arquivado['status'] == 'Arquivado por decurso de prazo'].shape[0]
print("\nQuantidade de solicitações para Minas Gerais com status 'Arquivado por decurso de prazo':", solicitacoes_mg_arquivado)


Quantidade de solicitações para Minas Gerais com status 'Arquivado por decurso de prazo': 302


In [39]:
# 3) Seu código aqui
df_bahia = consultar_api('BA', 'Município')


if 'interessado' in df_bahia.columns:
    municipio_mais_solicitacoes_deferidas = df_bahia[df_bahia['status'] == 'Deferido']['interessado'].value_counts().idxmax()
    print("\nInteressado (Município) na Bahia com mais solicitações deferidas:", municipio_mais_solicitacoes_deferidas)




Interessado (Município) na Bahia com mais solicitações deferidas: Bahia


In [40]:
# 4) Seu código aqui
df_bahia_estado = consultar_api('BA', 'Estado')
df_bahia_estado.to_csv('solicitacoes_bahia_estado.csv', index=False)
print("\nArquivo 'solicitacoes_bahia_estado.csv' salvo com sucesso.")


Arquivo 'solicitacoes_bahia_estado.csv' salvo com sucesso.
